In [1]:
import pandas as pd
import numpy as np
import datetime

In [44]:
def get_price(df, date, start):
    data = df.loc[df.date == date].close.to_numpy()
    if len(data) == 0:
        return -1
    if start:
        price = data[0]
    else:
        price = data[-1]
    return price

In [64]:
df_tweet = pd.read_csv('../data/twitter/tweets.csv', header=0, dtype=object)
df_goog = pd.read_csv('../data/prices/GOOG.csv', header=0)
df_tweet = df_tweet[df_tweet.included_symbols != '[]']

b = ['Apr', 'Mar', 'Feb', 'Jan']
monthDict0 = {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun', 
            7:'Jul', 8:'Aug', 9:'Sep', 10:'Oct', 11:'Nov', 12:'Dec'}
monthDict = dict((y,x) for x,y in monthDict0.items())
# need to reconvert timestamps
timestamps = df_tweet.timestamp.to_numpy()
t = np.zeros_like(timestamps)
p_change = np.zeros_like(timestamps)
for i, time in enumerate(timestamps):
    _, month, day, hms = time.split()
    if month in b:
        year = 2021
    else:
        year = 2020
    
    new_time = '%s-%s-%s %s'%(year, monthDict[month], day, hms)
    time_obj = datetime.datetime.strptime(new_time, '%Y-%m-%d %H:%M:%S')
    next_day_obj = time_obj + datetime.timedelta(days=1)
    curr_day = time_obj.strftime('%Y-%m-%d')
    next_day = next_day_obj.strftime('%Y-%m-%d')
    
    curr_price = get_price(df_goog, ' ' + curr_day, False)
    next_price = get_price(df_goog, ' ' + next_day, True)

    # find %change
    if curr_price or next_price == -1:
        change = -1
    else:
        change = ((next_price - curr_price) / next_price) * 100
    p_change[i] = change


    t[i] = int(time_obj.timestamp())

df_tweet.timestamp = t
df_tweet['p_change'] = p_change
# df_s = pd.to_datetime(df_tweet['timestamp'])
# res = pd.merge(df_tweet, df_goog, how="inner", on="timestamp")
# res.head()
df_tweet.to_csv('../data/twitter/tweet_change.csv')

In [40]:
df_goog = pd.read_csv('../data/prices/GOOG.csv', header=0)
# df_goog.date = df_goog.date.astype('str')
# df_goog.loc[df_goog.date == ' 2021-02-16'].close.to_numpy()
now = datetime.datetime.now() # current date and time

year = now.strftime("%Y-%m")
year

'2021-05'

In [65]:
df = pd.read_csv('../data/twitter/tweets.csv', header=0, dtype=object)
df.head()
df = df[df.included_symbols != '[]']
df.to_csv('../data/twitter/cleaned.csv')

In [9]:
df = pd.read_csv('../data/final_new_data/final_tweet_data.csv')
cols = ['BTC_returns', 'DOGE_returns', 'TSLA_returns', 'GOOG_returns', 'AAPL_returns', 'GME_returns', 'SPY_returns']
users = df.user.unique()
for user in users:
    tmp = df[df.user == user]
    tmp.to_csv('../data/final_new_data/%s.csv'%(user))
